This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-model-deploy-and-serve/xgboost-upload-tutorials).

## XGB RF Classifier Example

The following tutorial demonstrates deploying and serving an XGB RF Classifier model to Wallaroo.

The following XGBoost model types are supported by Wallaroo.  XGBoost models **not** supported by Wallaroo are supported via the [Arbitrary Python models, also known as Bring Your Own Predict (BYOP)](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/).

| XGBoost Model Type | Wallaroo Auto Packaging Supported |
|---|---|
| XGBClassifier | √ |
| XGBRegressor | √ |
| Booster Classifier | √ |
| Booster Classifier | √ |
| Booster Regressor | √ |
| Booster Random Forest Regressor | √ |
| Booster Random Forest Classifier | √ |
| XGBRFClassifier | √ |
| XGBRFRegressor | √ |
| XGBRanker\* | X |

* XGBRanker XGBoost models are currently supported via converting them to [BYOP](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-deploy/wallaroo-model-operations-upload-register/wallaroo-model-operations-upload-register-arbitrary-python/) models.

## Goal

Upload, deploy, and serve a sample XGB RF Classifier model.

### Resources

This tutorial provides the following:

* Models:
  * `./models/xgb_rf_classifier.pkl`: The sample XGBoost model that receives the `sklearn.datasets.load_iris` dataset.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
* A X64 Docker deployment to deploy the model on an edge location.
* The notebook "Wallaroo Run Anywhere Model Drift Observability with Assays: Preparation" has been run, and the model edge deployments executed.

## Steps

### Import Libraries

The first step is to import the libraries we will need.  See `./requirements.txt` for a list of additional libraries used with this tutorial.

In [1]:
import wallaroo
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.pipeline import Pipeline
import pyarrow as pa
from wallaroo.framework import Framework

import pickle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from xgboost import train, DMatrix

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  If logging in externally, update the `wallarooPrefix` and `wallarooSuffix` variables with the proper DNS information.  For more information on Wallaroo DNS settings, see the [Wallaroo DNS Integration Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-dns-guide/).

In [2]:
wl = wallaroo.Client()

### Set Variables and Helper Functions

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.

We'll set up some helper functions that will either use existing workspaces and pipelines, or create them if they do not already exist.

In [7]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

workspace_name = f'xgboost-rf-classifier'
pipeline_name = f'xgboost-rf-classifier'

model_name = 'xgboost-rf-classifier'
model_file_name = './models/xgb_rf_classifier.pkl'

### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline to deploy our model.

In [4]:
workspace = get_workspace(workspace_name, wl)
wl.set_current_workspace(workspace)

pipeline = wl.build_pipeline(pipeline_name)

### Upload XGBoost Model

XGBoost models are uploaded to Wallaroo through the `wallaroo.client.Client.upload_model` method.

#### Upload XGBoost Model Parameters

The following parameters are required for XGBoost models.  Note that while some fields are considered as **optional** for the `upload_model` method, they are required for proper uploading of a XGBoost model to Wallaroo.

| Parameter | Type | Description |
|---|---|---|
|`name` | `string` (*Required*) | The name of the model.  Model names are unique per workspace.  Models that are uploaded with the same name are assigned as a new **version** of the model. |
|`path` | `string` (*Required*) | The path to the model file being uploaded. 
|`framework` |`string` (*Required*) | Set as the `Framework.XGBOOST`. |
|`input_schema` | `pyarrow.lib.Schema` (*Required*) | The input schema in Apache Arrow schema format. |
|`output_schema` | `pyarrow.lib.Schema` (*Required*) | The output schema in Apache Arrow schema format. |
| `convert_wait` | `bool` (*Optional*) (*Default: True*) | <ul><li>**True**: Waits in the script for the model conversion completion.</li><li>**False**:  Proceeds with the script without waiting for the model conversion process to display complete. |

Once the upload process starts, the model is containerized by the Wallaroo instance.  This process may take up to 10 minutes.

#### Upload XGBoost Model Return

The following is returned with a successful model upload and conversion.

| Field | Type | Description |
|---|---|---|
| `name` | string | The name of the model. |
| `version` | string | The model version as a unique UUID. |
| `file_name` | string | The file name of the model as stored in Wallaroo. |
| `image_path` | string | The image used to deploy the model in the Wallaroo engine. |
| `last_update_time` | DateTime | When the model was last updated. |#

### Configure Input and Output Schemas

First we configure the input and output schemas in PyArrow format.

In [5]:
input_schema = pa.schema([
    pa.field('inputs', pa.list_(pa.float32(), list_size=4))
])

output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
    pa.field('probabilities', pa.list_(pa.float32(), list_size=3))
])

### Upload Model Example

With the input and output schemas defined, we now upload the XGBoost model.

In [8]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=Framework.XGBOOST, 
                        input_schema=input_schema, 
                        output_schema=output_schema)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a native runtime.
Model is attempting loading to a native runtime...incompatible

Model is pending loading to a container runtime.
Model is attempting loading to a container runtime.........successful

Ready


Name,xgboost-rf-classifier
Version,461ca493-2a59-4570-8e36-e741293bf133
File Name,xgb_rf_classifier.pkl
SHA,1e5ecfd77cb3bb75eab7aaf5b3975d037191c2d9e825805bff027fbec76be296
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.1.0-main-4806
Architecture,None
Acceleration,None
Updated At,2024-28-Mar 14:53:06


### Deploy Pipeline

With the model uploaded and packaged, we add the model as a pipeline model step.  For our deployment, we will set a minimum deployment configuration - this is the amount of resources the deployed pipeline uses from the cluster.

Once set, we deploy the pipeline, which allocates the assigned resources for the cluster and makes it available for inference requests.

In [9]:
pipeline.add_model_step(model)

deployment_config = DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .build()

In [10]:
pipeline.undeploy()
pipeline.deploy(deployment_config=deployment_config)
pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.28.1.194',
   'name': 'engine-599fcb5ff7-l8zgk',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'xgboost-rf-classifier',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'config': {'batch_config': None,
       'filter_threshold': None,
       'id': 67,
       'input_schema': '/////7AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAEAAAAEAAAAyP///wAAARAUAAAAJAAAAAQAAAABAAAAMAAAAAYAAABpbnB1dHMAAAAABgAIAAQABgAAAAQAAAAQABQACAAGAAcADAAAABAAEAAAAAAAAQMQAAAAHAAAAAQAAAAAAAAABAAAAGl0ZW0AAAYACAAGAAYAAAAAAAEAAAAAAA==',
       'model_version_id': 36,
       'output_schema': '//////AAAAAQAAAAAAAKAAwABgAFAAgACgAAAAABBAAMAAAACAAIAAAABAAIAAAABAAAAAIAAACEAAAABAAAAJT///8AAAEQFAAAACgAAAAEAAAAAQAAACQAAAANAAAAcHJvYmFiaWxpdGllcwAGAAgABAAGAAAAAwAAAND///8AAAEDEAAAABwAAAAEAAAAAAAAAAQAAABpdGVtAAAGAAgABgAGAAAAAAABABAAFAAIAAYABwAMAAAAEAAQAAAAAAABAhAAAAAkAAAABAAAAAA

### Run Sample Inference

The dataset is from the `sklearn.datasets.load_iris` examples.  These are converted to a pandas DataFrame, that is submitted to the deployed model in Wallaroo for an inference request.

In [11]:
dataset = load_iris()
X, y = dataset.data, dataset.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
dtrain = DMatrix(X_train, label=y_train)
dtest = DMatrix(X_test, label=y_test)

data = dtest.get_data().todense()[:100]

In [12]:
import pandas as pd

dataframe = pd.DataFrame({"inputs": data.tolist()})

results = pipeline.infer(dataframe)
results

,time,in.inputs,out.predictions,out.probabilities,anomaly.count
0,2024-03-28 14:53:24.507,"[6.0999999046, 2.7999999523, 4.6999998093, 1.2...",1,"[0.08693512, 0.8248159, 0.08824897]",0
1,2024-03-28 14:53:24.507,"[5.6999998093, 3.7999999523, 1.7000000477, 0.3...",0,"[0.82590103, 0.087049514, 0.08704949]",0
2,2024-03-28 14:53:24.507,"[7.6999998093, 2.5999999046, 6.9000000954, 2.2...",2,"[0.087590024, 0.087590046, 0.82481986]",0
3,2024-03-28 14:53:24.507,"[6.0, 2.9000000954, 4.5, 1.5]",1,"[0.08704949, 0.82590103, 0.08704949]",0
4,2024-03-28 14:53:24.507,"[6.8000001907, 2.7999999523, 4.8000001907, 1.3...",1,"[0.114213124, 0.66801673, 0.21777017]",0
5,2024-03-28 14:53:24.507,"[5.4000000954, 3.4000000954, 1.5, 0.400000006]",0,"[0.82590103, 0.087049514, 0.08704949]",0
6,2024-03-28 14:53:24.507,"[5.5999999046, 2.9000000954, 3.5999999046, 1.2...",1,"[0.08704949, 0.82590103, 0.08704949]",0
7,2024-03-28 14:53:24.507,"[6.9000000954, 3.0999999046, 5.0999999046, 2.2...",2,"[0.088018894, 0.08931566, 0.8226654]",0
8,2024-03-28 14:53:24.507,"[6.1999998093, 2.2000000477, 4.5, 1.5]",1,"[0.08900264, 0.8182466, 0.09275077]",0
9,2024-03-28 14:53:24.507,"[5.8000001907, 2.7000000477, 3.9000000954, 1.2...",1,"[0.08704949, 0.82590103, 0.08704949]",0


### Undeploy the Pipeline

With the tutorial complete, we undeploy the pipeline and return the resources back to the cluster.

In [13]:
pipeline.undeploy()

name,xgboost-rf-classifier
created,2024-03-28 14:50:35.483627+00:00
last_updated,2024-03-28 14:53:08.748796+00:00
deployed,False
arch,None
accel,None
tags,
versions,"7b9e7601-0acb-4e25-8180-ce1c1649044d, 180e2702-1f9f-46f3-af66-df84427f3667"
steps,xgboost-rf-classifier
published,False
